In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)

True

In [3]:
from nltk.corpus import wordnet

In [4]:
nltk.download('averaged_perceptron_tagger',quiet=True)

True

In [19]:
#!pip install contractions

In [23]:
import csv
import pandas as pd
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
# from keras.layers.embeddings import Embedding

## Plot
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import word_tokenize



# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn import model_selection, preprocessing, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.svm import SVC
from keras.models import load_model
import pickle
# Sampling
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

import contractions
import re, string, unicodedata
import nltk
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping

# #wordcloud
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

In [22]:
!pip install inflect

  Obtaining dependency information for inflect from https://files.pythonhosted.org/packages/fb/c6/d9feb758be584f729424390af24687d3a4363d968164f94079f83cd536b4/inflect-7.0.0-py3-none-any.whl.metadata


In [ ]:
# The code starts by importing necessary libraries, setting up Google Drive, and downloading required NLTK resources.
# It then imports additional libraries for text processing, machine learning, and handling imbalanced datasets.
# The read_bias_data function is defined to read bias data from two CSV files, drop unnecessary columns, and concatenate relevant text columns.

In [ ]:
# The label_encode function is defined to encode bias labels with numerical values for machine learning models.

In [24]:
def read_bias_data(f1, f2):
  cols=["title", "text", "sen_obj", "source", "bias"]
  df = pd.read_csv(f1, skiprows=1, dtype=str, names=cols)
  tdf = pd.read_csv(f2, skiprows=1, dtype=str, names=cols)
  df=df.drop(["sen_obj"], 1)
  tdf=tdf.drop(["sen_obj"], 1)
  df["all"]=df.source + " " + df.title + " "+ df.text
  tdf["all"]=tdf.source + " " + tdf.title + " "+ tdf.text
  df=df.dropna()
  tdf=tdf.dropna()
  return df, tdf

def label_encode(df, tdf):
  tdf["bias"]= tdf["bias"].replace("lc", 1)
  tdf["bias"]= tdf["bias"].replace("rc", 2)
  tdf["bias"]= tdf["bias"].replace("left", 1)
  tdf["bias"]= tdf["bias"].replace("right", 2)
  tdf["bias"]= tdf["bias"].replace("least-biased",0)
  df["bias"]= df["bias"].replace("1", 1)
  df["bias"]= df["bias"].replace("2", 2)
  df["bias"]= df["bias"].replace("0", 0)
  return df, tdf

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Use get() with a default value



In [ ]:
# Two functions, get_wordnet_pos and clean_text, are defined for part-of-speech tagging and text cleaning.

In [25]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove digits
    text = re.sub(r'\d', '', text)

    # Remove stop words
    stops = set(stopwords.words("english"))
    text = ' '.join([word for word in word_tokenize(text) if word.lower() not in stops])

    # Use BeautifulSoup to handle HTML entities
    text = BeautifulSoup(text, "html.parser").get_text()

    # Fix contractions
    text = contractions.fix(text)

    # Handle specific contractions and substitutions
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmed_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in word_tokenize(text)]
    text = " ".join(lemmed_words)

    return text


In [26]:
# def read_clickbait_data(data):
#   # clickbait_data = pd.read_json(open(fi, "r", encoding="utf8"), lines=True)
#   # clickbait_data1 = pd.read_json(open(ft, "r", encoding="utf8"), lines=True)
#   # data = pd.merge(clickbait_data, clickbait_data1, on = 'id')
#   labels = data.truthClass.map(lambda x : 1 if x=='clickbait' else 0)
#   df = pd.DataFrame({'label':labels, 'text':data.postText})
#   df['text'] = df['text'].apply(lambda x: ','.join(map(str, x)))
#   return df

In [27]:
df = pd.read_csv('clickbait_training.csv')

In [28]:
df

,label,text
0,0,Apple's iOS 9 'App thinning' feature will give...
1,0,RT @kenbrown12: Emerging market investors are ...
2,1,U.S. Soccer should start answering tough quest...
3,0,How theme parks like Disney World left the mid...
4,1,Could light bulbs hurt your health? One compan...
...,...,...
2454,1,Man who received world's first penis transplan...
2455,0,"RT @NYTSports: Abby didn't start, team couldn'..."
2456,0,Obama defends Affordable Care Act ahead of Sup...
2457,0,New study of the Deflategate report concludes ...


In [29]:
df["all_cleaned"]=df["text"].apply(clean_text)

In [30]:
df

,label,text,all_cleaned
0,0,Apple's iOS 9 'App thinning' feature will give...,apple io app thin feature give phone storage b...
1,0,RT @kenbrown12: Emerging market investors are ...,rt kenbrown emerge market investor best monty ...
2,1,U.S. Soccer should start answering tough quest...,u soccer start answer tough question hope solo...
3,0,How theme parks like Disney World left the mid...,theme park like disney world left middle class...
4,1,Could light bulbs hurt your health? One compan...,could light bulb hurt health one company put w...
...,...,...,...
2454,1,Man who received world's first penis transplan...,man receive world first penis transplant becom...
2455,0,"RT @NYTSports: Abby didn't start, team couldn'...",rt nytsports abby do not start team could not ...
2456,0,Obama defends Affordable Care Act ahead of Sup...,obama defends affordable care act ahead suprem...
2457,0,New study of the Deflategate report concludes ...,new study deflategate report concludes deeply ...


In [31]:
from sklearn.model_selection import GridSearchCV
Cs = [0.001, 0.01, 0.1, 1, 10]
tfidf_vect = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))


In [32]:
# Data is read from a CSV file, and text cleaning is applied to the "text" column.
# Grid search is used to find the best parameters for a Support Vector Machine (SVM) model using the TF-IDF representation of the text.

In [33]:
X=tfidf_vect.fit_transform(df.text)
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas, 'kernel': ['rbf', 'linear', 'poly']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X, df.label)
grid_search.best_params_


{'C': 10, 'gamma': 0.001, 'kernel': 'linear'}

In [34]:
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
svc = SVC(C= 10, gamma= 0.001, kernel= 'linear')
# svc= SVC()

In [35]:
ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777), svc)
SMOTE_pipeline = make_pipeline(tvec, SMOTE(random_state=777), svc)

In [ ]:
# Two pipelines are created for oversampling (RandomOverSampler and SMOTE) combined with an SVM classifier.
# The SMOTE pipeline is fitted to the training data, and the resulting model is saved using pickle.

In [36]:
SMOTE_pipeline.fit(df.text, df.label)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=100000, ngram_range=(1, 3))),
                ('smote', SMOTE(random_state=777)),
                ('svc', SVC(C=10, gamma=0.001, kernel='linear'))])

In [37]:
pickle.dump(SMOTE_pipeline, open("model_clickbait_final2.sav", 'wb'))
loaded_model = pickle.load(open("model_clickbait_final2.sav", 'rb'))

In [38]:
def evaluate_clickbait_pip(pipeline, tdf):
  print("Score: ",pipeline.score(tdf.text, tdf.label))
  y_pred=pipeline.predict(tdf.text)
  print(confusion_matrix(tdf.label , y_pred))
  print(classification_report(tdf.label, y_pred))
def predict_clickbait(pipeline, tdf):
  out={0:"Non-Clickbait", 1:"Clickbait"}
  pred=out[pipeline.predict(tdf.text)[0]]
  print("The article is ", pred)


In [ ]:
# Two functions, evaluate_clickbait_pip and predict_clickbait, are defined for evaluating the model and making predictions.

In [ ]:
# Test data is read, text cleaning is applied, and the evaluate_clickbait_pip function is called to assess the model's performance on the test data.

In [39]:
tdf = pd.read_csv("Clickbait_testing.csv")

In [40]:
tdf

,label,text
0,1,"""Uncover the hidden secrets to eternal happine..."
1,1,"""These mind-blowing weight loss hacks will tra..."
2,1,"""The shocking truth about celebrities' secret ..."
3,1,"""Discover the amazing power of positive thinki..."
4,1,"""You won't believe these money-making strategi..."
...,...,...
194,0,"""The role of education in promoting environmen..."
195,0,"""Analyzing the impact of social media on commu..."
196,0,"""Tips for effective time management for entrep..."
197,0,"""A comprehensive guide to understanding princi..."


In [41]:
tdf["all_cleaned"]=tdf["text"].apply(clean_text)


In [42]:
tdf

,label,text,all_cleaned
0,1,"""Uncover the hidden secrets to eternal happine...",uncover hidden secret eternal happiness will n...
1,1,"""These mind-blowing weight loss hacks will tra...",mindblowing weight loss hack transform life se...
2,1,"""The shocking truth about celebrities' secret ...",shock truth celebrity secret life reveal mustread
3,1,"""Discover the amazing power of positive thinki...",discover amaze power positive think success li...
4,1,"""You won't believe these money-making strategi...",will not believe moneymaking strategy actually...
...,...,...,...
194,0,"""The role of education in promoting environmen...",role education promote environmental conservation
195,0,"""Analyzing the impact of social media on commu...",analyze impact social medium community engagement
196,0,"""Tips for effective time management for entrep...",tip effective time management entrepreneur bus...
197,0,"""A comprehensive guide to understanding princi...",comprehensive guide understand principle human...


In [43]:
evaluate_clickbait_pip(SMOTE_pipeline, tdf )

Score:  0.8844221105527639
[[93  7]
 [16 83]]
              precision    recall  f1-score   support

           0       0.85      0.93      0.89       100
           1       0.92      0.84      0.88        99

    accuracy                           0.88       199
   macro avg       0.89      0.88      0.88       199
weighted avg       0.89      0.88      0.88       199

